In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -raw_data_preprocessing -pandas -numpy

In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.append("../")
sys.path.append("../ukbb_preprocessing/")

from raw_data_preprocessing.raw_data_loader import raw_data_loader
from raw_data_preprocessing.constants import *

from utils import rename_variables, DataRegisterer, percentile

REGISTER = False

loader = raw_data_loader()
registerer = DataRegisterer()

data_asset = loader.ws.data.get(name="clin_ascvd", version="6")

In [ ]:
df = pd.read_csv(data_asset.path)
prs_df = rename_variables(df)
df.set_index('IID', inplace=True)
cols = prs_df.columns.str.startswith('eprs') | prs_df.columns.str.startswith('prs') | prs_df.columns.str.startswith('ascvd_')
prs_df = prs_df.loc[:, cols]

# QC

In [ ]:
# compute auc
from sklearn.metrics import roc_auc_score

for disease in ["cad", "cvd", "ht", "iss"]:
    print(disease.upper())
    for prs in ["prs", "eprs"]:

        y_true = df.ascvd_10yr_label.values
        y_score = prs_df[f"{prs}_{disease}"].values

        mask = ~np.isnan(y_true) & ~np.isnan(y_score)
        y_true = y_true[mask]
        y_score = y_score[mask]

        print(f"\t{prs.upper()}:\t{roc_auc_score(y_true, y_score)}")

# Save

In [ ]:
import os

splits = loader._load_csv_folder(data_asset_name="ukbb_golden_splits_by_diagnosis_date", version=3)
splits = [split.set_index(split.IID.astype(int)) for split in splits]

for disease in ["cad", "cvd", "ht", "iss"]:

    fname = f"prs_{disease}"
    os.makedirs(fname, exist_ok=True)

    score = prs_df[[fname]].copy()
    score["y_score"] = score[f"prs_{disease}"]

    threshold = score['y_score'].aggregate(percentile(0.95))

    score["y_pred"] = (score["y_score"] > threshold).astype(int)

    score = score[['y_score', 'y_pred']]

    for i, s in enumerate(splits):
        s["IID"] = s["IID"].astype(int)
        s = s.set_index("IID").merge(score, on="IID")
        s.to_csv(f'{fname}/npx_clin_ascvd_{i}_test.csv', sep=',')